In [3]:
import sframe

# Read Data
products = sframe.SFrame('logistic-regression/data/')

In [4]:
def remove_punctuation(text):
    import string
    return text.translate(None, string.punctuation)

In [5]:
# Remove Punctuation
products['review_clean'] = products['review'].apply(remove_punctuation)

# Remove NA
products = products.fillna('review', "")

# Ignore reviews with rating = 3
products = products[products['rating'] != 3]

# Change Rating = 4 or 5 to be positive
products['sentiment'] = products['rating'].apply(lambda rating: +1 if rating > 3 else -1)

In [6]:
# Split into training set and testing set
train_data, test_data = products.random_split(.8, seed=1)

In [7]:
# Bag of Words Model
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
train_matrix = vectorizer.fit_transform((train_data['review_clean']))
test_matrix = vectorizer.transform(
    (test_data['review_clean']))  # Test data must be transform in the same way as the training data

In [8]:
from sklearn.linear_model import LogisticRegression

sentiment_model = LogisticRegression()
sentiment_model.fit(train_matrix, train_data['sentiment'])
params = sentiment_model.get_params()
print sum(1 for key in params if params[key] >= 0)

In [10]:
# Making Predictions
sample_test_data = test_data[10:13]
sample_test_matrix = vectorizer.transform(sample_test_data['review_clean'])
scores = sentiment_model.decision_function(sample_test_matrix)
print scores
prob = sentiment_model.predict_proba(sample_test_matrix)
print prob


[  5.60223347  -3.17035343 -10.42246382]
[[  3.67605066e-03   9.96323949e-01]
 [  9.59703255e-01   4.02967451e-02]
 [  9.99970244e-01   2.97555890e-05]]
